In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import OneHotEncoder as OHE
from matplotlib.patches import Ellipse
import math
import matplotlib.pyplot as plt
from scipy import stats
import scipy.sparse as sparse
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time

In [2]:
df = pd.read_csv(r"taxi_17.csv")
df.head()

,Unnamed: 0,start_time,end_time,trip_seconds,trip_miles,start_census,end_census,start_community,end_community,fare,...,Dropoff Centroid Location,new_TaxiID,dur_min,date,weekday,avg_temp,precip,hour,day,month
0,0,2017-01-01 00:00:00,2017-01-01 00:30:00,1800.0,0.0,1.703108e+10,1.703108e+10,8,8,13.25,...,POINT (-87.6262149064 41.8925077809),2751,30.0,2017-01-01 00:00:00,6,-0.6,0.0,0,1,1
1,1,2017-01-01 00:00:00,2017-01-01 00:00:00,660.0,2.7,NaN,NaN,2,3,10.00,...,POINT (-87.6558787862 41.96581197),5058,11.0,2017-01-01 00:00:00,6,-0.6,0.0,0,1,1
2,2,2017-01-01 00:00:00,2017-01-01 00:15:00,120.0,0.5,NaN,NaN,16,16,4.50,...,POINT (-87.7234523905 41.9535821253),5047,2.0,2017-01-01 00:00:00,6,-0.6,0.0,0,1,1
3,3,2017-01-01 00:00:00,2017-01-01 00:00:00,240.0,1.3,1.703184e+10,1.703183e+10,32,28,6.25,...,POINT (-87.6572331997 41.8852813201),2416,4.0,2017-01-01 00:00:00,6,-0.6,0.0,0,1,1
4,4,2017-01-01 00:00:00,2017-01-01 00:00:00,420.0,0.0,1.703128e+10,1.703184e+10,28,32,6.00,...,POINT (-87.6327464887 41.8809944707),3952,7.0,2017-01-01 00:00:00,6,-0.6,0.0,0,1,1


In [3]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
work_data = df.loc[:, ("start_census","weekday","hour","day","month","avg_temp")]
work_data

,start_census,weekday,hour,day,month,avg_temp
0,1.703108e+10,6,0,1,1,-0.6
1,NaN,6,0,1,1,-0.6
2,NaN,6,0,1,1,-0.6
3,1.703184e+10,6,0,1,1,-0.6
4,1.703128e+10,6,0,1,1,-0.6
...,...,...,...,...,...,...
21737161,NaN,6,23,31,12,-12.8
21737162,1.703108e+10,6,23,31,12,-12.8
21737163,1.703124e+10,6,23,31,12,-12.8
21737164,NaN,6,23,31,12,-12.8


In [5]:
work_data = work_data.dropna()
work_data

,start_census,weekday,hour,day,month,avg_temp
0,1.703108e+10,6,0,1,1,-0.6
3,1.703184e+10,6,0,1,1,-0.6
4,1.703128e+10,6,0,1,1,-0.6
5,1.703108e+10,6,0,1,1,-0.6
6,1.703184e+10,6,0,1,1,-0.6
...,...,...,...,...,...,...
21737159,1.703108e+10,6,23,31,12,-12.8
21737160,1.703108e+10,6,23,31,12,-12.8
21737162,1.703108e+10,6,23,31,12,-12.8
21737163,1.703124e+10,6,23,31,12,-12.8


In [6]:
def create_dataset(data, resolution=4):
    df = data.copy()

    df['new_hour'] = pd.cut(df['hour'], bins=range(-1, 25, resolution), labels=False)
    df = df.drop('hour', axis=1)
    df['new_hour'] = df['new_hour'].astype(int)
    
    
    df2 = df.loc[:, ("start_census","weekday","new_hour","day","month","avg_temp")]
    df2['occurrences'] = df2.groupby(["start_census","weekday","new_hour","day","month"]).transform('count')
    return df2

In [7]:
def convert_sparse_matrix_to_sparse_tensor(X):
    coo = X.tocoo()
    indices = np.mat([coo.row, coo.col]).transpose()
    return tf.SparseTensor(indices, coo.data, coo.shape)

In [8]:
res_work_data = create_dataset(work_data, resolution=2)
res_work_data = res_work_data.sample(frac=1)
res_work_data.drop_duplicates(inplace=True)
res_work_data

,start_census,weekday,new_hour,day,month,avg_temp,occurrences
19416590,1.703108e+10,2,8,15,11,8.05,484
6982137,1.703132e+10,2,3,26,4,20.00,336
14568704,1.703128e+10,3,9,24,8,20.60,317
19709898,1.703198e+10,0,9,20,11,10.00,230
16665186,1.703128e+10,6,1,1,10,15.00,48
...,...,...,...,...,...,...,...
9039897,1.703184e+10,4,4,26,5,11.70,1
14797853,1.703103e+10,1,4,29,8,17.20,4
7986790,1.703116e+10,2,9,10,5,18.30,1
7410923,1.703106e+10,1,8,2,5,7.20,1


In [9]:
# apply normalization techniques by Column 1
column = 'avg_temp'
res_work_data[column] = (res_work_data[column] - res_work_data[column].min()) / (res_work_data[column].max() - res_work_data[column].min())    
  
# view normalized data
display(res_work_data)

,start_census,weekday,new_hour,day,month,avg_temp,occurrences
19416590,1.703108e+10,2,8,15,11,0.504596,484
6982137,1.703132e+10,2,3,26,4,0.724265,336
14568704,1.703128e+10,3,9,24,8,0.735294,317
19709898,1.703198e+10,0,9,20,11,0.540441,230
16665186,1.703128e+10,6,1,1,10,0.632353,48
...,...,...,...,...,...,...,...
9039897,1.703184e+10,4,4,26,5,0.571691,1
14797853,1.703103e+10,1,4,29,8,0.672794,4
7986790,1.703116e+10,2,9,10,5,0.693015,1
7410923,1.703106e+10,1,8,2,5,0.488971,1


In [10]:
train, test = train_test_split(res_work_data, test_size=0.2)

In [11]:
one_hot_data = res_work_data.loc[:, ("start_census","weekday","new_hour","day","month")]

In [12]:
OHEt = OHE().fit(one_hot_data.to_numpy())

In [13]:
train_OHE = OHEt.transform(train.loc[:, ("start_census","weekday","new_hour","day","month")].to_numpy())
test_OHE = OHEt.transform(test.loc[:, ("start_census","weekday","new_hour","day","month")].to_numpy())

In [14]:
sp_train = sparse.hstack((train_OHE,train['avg_temp'].to_numpy().reshape(-1, 1)))
sp_test = sparse.hstack((test_OHE,test['avg_temp'].to_numpy().reshape(-1, 1)))
#sp_train = convert_sparse_matrix_to_sparse_tensor(sp_train)
sp_test = convert_sparse_matrix_to_sparse_tensor(sp_test)
sp_train = sparse.csr_matrix(sp_train)

In [15]:
#y_test = sparse.csr_matrix(test.loc[:, ("occurrences")].to_numpy().reshape(-1, 1))
#y_test = convert_sparse_matrix_to_sparse_tensor(y_test)
#y_test = tf.sparse.to_dense(y_test)
#y_test

In [16]:
y_train = sparse.csr_matrix(train.loc[:, ("occurrences")].to_numpy().reshape(-1, 1))
y_test = sparse.csr_matrix(test.loc[:, ("occurrences")].to_numpy().reshape(-1, 1))
#y_train = convert_sparse_matrix_to_sparse_tensor(y_train)
y_test = convert_sparse_matrix_to_sparse_tensor(y_test)

In [17]:
#sp_train = tf.sparse.reorder(sp_train)
#y_train = tf.sparse.reorder(y_train)

In [18]:
sp_test = tf.sparse.reorder(sp_test)
y_test = tf.sparse.reorder(y_test)
sp_test = tf.sparse.to_dense(sp_test)
y_test = tf.sparse.to_dense(y_test)

In [25]:
network = models.Sequential()

network.add(layers.Dense(617, activation='relu', input_shape=(617,)))
network.add(layers.Dense(617, activation='relu', input_shape=(617,)))
network.add(layers.Dense(617, activation='relu', input_shape=(617,)))
network.add(layers.Dense(617, activation='relu', input_shape=(617,)))
network.add(layers.Dense(512, activation='relu', input_shape=(617,)))

network.add(layers.Dense(1, activation='linear'))


In [26]:
network.compile(optimizer='SGD',loss='mean_absolute_error')

In [21]:
#test_sp_train = tf.sparse.slice(sp_train,[0,0],[256, 617])
#test_y_train = tf.sparse.slice(y_train,[0,0],[256, 1])
#test_sp_train = tf.sparse.reorder(test_sp_train)
#test_y_train = tf.sparse.reorder(test_y_train)
#test_sp_train = tf.sparse.to_dense(test_sp_train)
#test_y_train = tf.sparse.to_dense(test_y_train)
#print(test_sp_train,test_y_train)

In [22]:
#test_sp_train=sp_train[0:255,:]
#test_sp_train = test_sp_train.todense()
#test_sp_train.shape

In [27]:
b_s = 256
for i in range(10000):
    if i%100 == 0 and i!=0:
        print(i)
    if i%1000 == 0:
        test_loss = network.evaluate(sp_test, y_test)
    # Traing the network
    test_sp_train = sp_train[i*b_s-1:min(i*b_s+b_s-1,sp_train.shape[0]-1),:]
    test_y_train = y_train[i*b_s-1:min(i*b_s+b_s-1,y_train.shape[0]-1),:]
    #test_sp_train = tf.sparse.slice(sp_train,[i,0],[min(i+b_s-1, sp_train.shape[0]-1), 617])
    #test_y_train = tf.sparse.slice(y_train,[i,0],[min(i+b_s-1, y_train.shape[0]-1), 1])
    test_sp_train = test_sp_train.todense()
    test_y_train = test_y_train.todense()
    network.train_on_batch(test_sp_train, test_y_train)

2276/2276 [==============================] - 6s 3ms/step - loss: 80.7435
100
200
300
400
500
600
700
800
900
1000
2276/2276 [==============================] - 7s 3ms/step - loss: 23.7435
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2276/2276 [==============================] - 6s 3ms/step - loss: 29.5697
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
2276/2276 [==============================] - 6s 3ms/step - loss: 29.5697
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
2276/2276 [==============================] - 6s 3ms/step - loss: 29.5697
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
2276/2276 [==============================] - 6s 3ms/step - loss: 29.5697
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
2276/2276 [==============================] - 7s 3ms/step - loss: 29.5697
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
2276/2276 [==============================] - 6s 3ms/step - loss: 29.5697
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
2276/2276 [==============

In [24]:
test_loss = network.evaluate(sp_test, y_test)
print('test_err:', test_loss)

2276/2276 [==============================] - 5s 2ms/step - loss: 26.4233
test_err: 26.4233455657959
